## <font color=#B2D732> <span style="background-color: #4424D6"> Imports

In [16]:
import sys,json
import glob, os
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/code/")

from seed_to_voxels import Seed2voxels

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <font color=#B2D732> <span style="background-color: #4424D6"> A/ Initialization

 ### <font color=#4424D6> I. Configuration & parameters </font >

In [17]:
# Load config file ------------------------------------------------------------
with open('../../config/config_seed2voxels.json') as config_file: # the notebook should be in 'xx/notebook/' folder #config_proprio
    config = json.load(config_file) # load config file should be open first and the path inside modified
    #config['list_subjects']=config["list_subjects_younger"]
seed_indiv=False

### <font color=#4424D6> II. Initialize class based on this </font>

In [19]:
seed2voxels=Seed2voxels(config,seed_indiv) # initialize the function

Start the analysis on: 31 participants
targeted voxel's group mask: MNI_GM_2mm
/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//hc_project_analyses/masks/brain/MNI_GM_2mm.nii.gz
/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//hc_project_analyses/masks/spinalcord/levels/icaps/gm/icap_C1_V_gm.nii.gz
/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//hc_project_analyses/masks/spinalcord/levels/icaps/gm/icap_C2_V_gm.nii.gz
/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//hc_project_analyses/masks/spinalcord/levels/icaps/gm/icap_C3_V_gm.nii.gz
/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//hc_project_analyses/masks/spinalcord/levels/icaps/gm/icap_C4_V_gm.nii.gz
/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//hc_project_analyses/masks/spinalcord/levels/icaps/gm/icap_C5_V_gm.nii.gz
/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//hc_project_analyses/masks/spin

## <font color=#B2D732> <span style="background-color: #4424D6"> B/ Data extraction </font></span>
### <font color=#4424D6> I. Time series extraction - Target </font>


In [12]:
target_timeseries,seeds_timeseries = seed2voxels.extract_data(redo=True,n_jobs=8) 

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py", line 567, in run
    self.flag_executor_shutting_down()
  File "/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py", line 756, in flag_executor_shutting_down
    self.kill_workers()
  File "/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py", line 766, in kill_workers
    recursive_terminate(p)
  File "/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/joblib/externals/loky/backend/utils.py", line 28, in recursive_termina

KeyboardInterrupt: 

## <font color=#B2D732> <span style="background-color: #4424D6"> C/ Correlation analysis

In [96]:
output_dir={};output_file={}; 
for seed_name in config["seeds"]["seed_names"]:
    output_dir[seed_name]=config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/' + seed_name + '/' + config["targeted_voxels"]["target_name"]+ '_fc_maps/Corr/'
    if not os.path.exists(output_dir[seed_name]):
        os.mkdir(output_dir[seed_name])
    
    output_file[seed_name]=  output_dir[seed_name] +'/corr_' + str(len(config['list_subjects'])) + 'subjects_seed_' + seed_name + '_s'
    
    # run correlation analysis
    corr=seed2voxels.correlation_maps(seeds_timeseries["zmean"][seed_name],
                                      target_timeseries["zscored"],
                                      output_img=output_file[seed_name],
                                      Fisher=True,
                                      partial=False,
                                      save_maps=True,
                                      smoothing_output=None,
                                      redo=True,
                                      n_jobs=8)

    #calculate the mean across participant
    string="fslmaths " +output_file[seed_name] + " -Tmean " + output_file[seed_name].split(".")[0] + "_mean.nii.gz"
    os.system(string)
   

/home/kinany/anaconda3/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/kinany/anaconda3/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


## <font color=#B2D732> <span style="background-color: #4424D6"> D/ Mutual information

In [57]:
output_dir={};output_file={};
for seed_name in config["seeds"]["seed_names"]:
    output_dir[seed_name]=config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/' + seed_name + '/' + config["targeted_voxels"]["target_name"]+ '_fc_maps/MI/'
   
    if not os.path.exists(output_dir[seed_name]):
            os.mkdir(output_dir[seed_name])
    
    output_file[seed_name]=  output_dir[seed_name] +'/mi_' + str(len(config['list_subjects'])) + 'subjects_seed_' + seed_name + '_ss_z'
  
    
    mi=seed2voxels.mutual_info_maps(seeds_timeseries["zmean"][seed_name],
                                    target_timeseries["zscored"],
                                    output_img=output_file[seed_name],
                                    save_maps=True,
                                    smoothing_output=[6,6,6],redo=True, n_jobs=8)
    
    #calculate the mean across participant
    string="fslmaths " +output_file[seed_name] + " -Tmean " + output_file[seed_name].split(".")[0] + "_mean.nii.gz"
    os.system(string)

/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/M1infmidsup/C1C7_gmwm_fc_maps/MI/mi_sub-A007.nii.gz
/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/M1infmidsup/C1C7_gmwm_fc_maps/MI/mi_sub-P099.nii.gz
/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/M1infmidsup/C1C7_gmwm_fc_maps/MI/mi_sub-A009.nii.gz
/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/M1infmidsup/C1C7_gmwm_fc_maps/MI/mi_sub-P030.nii.gz
/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/M1infmidsup/C1C7_gmwm_fc_maps/MI/mi_sub-A011.nii.gz
/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/M1infmidsup/C1C7_gmwm_fc_maps/MI/mi_sub-A013.nii.gz
/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/M1infmidsup/C1C7_gmwm_fc_maps/MI/mi_sub-A008

## <font color=#B2D732> <span style="background-color: #4424D6"> D/ Distance correlation

In [14]:
output_dir={};output_file={};
for seed_name in config["seeds"]["seed_names"]:
    
    
    output_dir[seed_name]=config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/' + seed_name + '/' + config["targeted_voxels"]["target_name"]+ '_fc_maps/dCorr/'
   
    if not os.path.exists(output_dir[seed_name]):
            os.mkdir(output_dir[seed_name])
    
    output_file[seed_name]=  output_dir[seed_name] +'/dCorr_' + str(len(config['list_subjects'])) + 'subjects_seed_' + seed_name.split('_')[-1] + '_s_z'
  
    dist_corr=seed2voxels.nonlinear_maps(seeds_timeseries["zmean"][seed_name],
                                    target_timeseries["zscored"],
                                    metric="distCorr",
                                    output_img=output_file[seed_name],
                                    save_maps=True,
                                    smoothing_output=None,redo=True, n_jobs=8)
    
    #calculate the mean across participant
    string="fslmaths " +output_file[seed_name] + " -Tmean " + output_file[seed_name].split(".")[0] + "_mean.nii.gz"
    os.system(string)

/home/kinany/anaconda3/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
Distance correlation-based Estimator (mi_type=cc, implementation=dcor)
/home/kinany/anaconda3/lib/python3.9/site-packages/dcor/_fast_dcov_avl.py:554: UserWarning: Falling back to uncompiled AVL fast distance covariance terms because of TypeError exception raised: No matching definition for argument type(s) readonly array(float32, 1d, A), array(float64, 1d, C), bool. Rembember: only floating point values can be used in the compiled implementations.
  warnings.warn(


/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/icap_C1/MNI_GM_2mm_fc_maps/dCorr/mi_sub-A006.nii.gz


Distance correlation-based Estimator (mi_type=cc, implementation=dcor)


/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/icap_C2/MNI_GM_2mm_fc_maps/dCorr/mi_sub-A006.nii.gz


Distance correlation-based Estimator (mi_type=cc, implementation=dcor)


...

/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/icap_C5/MNI_GM_2mm_fc_maps/dCorr/mi_sub-A006.nii.gz


/home/kinany/anaconda3/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
Distance correlation-based Estimator (mi_type=cc, implementation=dcor)
/home/kinany/anaconda3/lib/python3.9/site-packages/dcor/_fast_dcov_avl.py:554: UserWarning: Falling back to uncompiled AVL fast distance covariance terms because of TypeError exception raised: No matching definition for argument type(s) readonly array(float32, 1d, A), array(float64, 1d, C), bool. Rembember: only floating point values can be used in the compiled implementations.
  warnings.warn(


/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/icap_C6/MNI_GM_2mm_fc_maps/dCorr/mi_sub-A006.nii.gz


Distance correlation-based Estimator (mi_type=cc, implementation=dcor)


/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/icap_C7/MNI_GM_2mm_fc_maps/dCorr/mi_sub-A006.nii.gz


## <font color=#B2D732> <span style="background-color: #4424D6"> F/ DTW

In [26]:
output_dir={};output_file={};
for seed_name in config["seeds"]["seed_names"]:
    output_dir[seed_name]=config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/' + seed_name + '/' + config["targeted_voxels"]["target_name"]+ '_fc_maps/DTW/'
   
    if not os.path.exists(output_dir[seed_name]):
            os.mkdir(output_dir[seed_name])
    
    output_file[seed_name]=  output_dir[seed_name] +'/dtw_' + str(len(config['list_subjects'])) + 'subjects_seed_' + seed_name + '_ss'
  
    
    dtw=seed2voxels.dtw_maps(seeds_timeseries["zmean"][seed_name],
                                    target_timeseries["zscored"],
                                    output_img=output_file[seed_name],
                                    save_maps=True,
                                    smoothing_output=None,redo=True, n_jobs=8)
    
    #calculate the mean across participant
    string="fslmaths " +output_file[seed_name] + " -Tmean " + output_file[seed_name].split(".")[0] + "_mean.nii.gz"
    os.system(string)

/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/C1C7_D_gmwm/MNI_GM_3mm_fc_maps/DTW/mi_sub-A006.nii.gz
/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/C1C7_D_gmwm/MNI_GM_3mm_fc_maps/DTW/mi_sub-A013.nii.gz
/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/C1C7_D_gmwm/MNI_GM_3mm_fc_maps/DTW/mi_sub-P030.nii.gz
/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/C1C7_D_gmwm/MNI_GM_3mm_fc_maps/DTW/mi_sub-P099.nii.gz
/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/C1C7_D_gmwm/MNI_GM_3mm_fc_maps/DTW/mi_sub-A009.nii.gz
/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/C1C7_D_gmwm/MNI_GM_3mm_fc_maps/DTW/mi_sub-A008.nii.gz
/media/miplab-nas2/Data3/BMPD/hc_project//brain_spine/results/seed_to_voxels//1_first_level/C1C7_D_gmwm/MNI_GM_3mm_fc_maps/D

#### <font color=#4424D6> Two sample t-test

In [ ]:
import shutil
#1. select the analyses to compared
#'spinalcord_GM-C4C5L','spinalcord_GM-C4C5R',
#'spinalcord_GM-C6C8L','spinalcord_GM-C6C8R'
        
seed1='spinalcord_GM-C6C8R'
seed2='spinalcord_GM-C6C8L'
output_dir=config['main_dir'] + config['seed2vox_dir'] + '/2_second_level/'+seed1+'-'+seed2.split('-')[-1]
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
#2. merge the images from the 2 contrasts in one 4D image
    run_proc('fslmerge -t {} {} {}'.format(output_dir + '/'+seed1+'-'+seed2.split('-')[-1]+'_images.nii', 
                                    config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/' +seed1 + '/' + target_name[0].split('_')[0]+ '_fc_maps/21subjects_seed_'+seed1.split('_')[-1]+'_s_zcorr',
                                    config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/' +seed2 + '/' + target_name[0].split('_')[0]+ '_fc_maps/21subjects_seed_'+seed2.split('_')[-1]+'_s_zcorr'))

#3. Run randomize
    run_proc('randomise -i {} -o {} -d {} -t {} -e {} -m {} -v 5 -n 500  -T'.format(output_dir + '/'+seed1+'-'+seed2.split('-')[-1]+'_images.nii',
                                                                                    output_dir + '/'+seed1+'-'+seed2.split('-')[-1],
                                                                                    config['main_dir'] + config['seed2vox_dir'] + '/2_second_level/fsl_designs/2paired-t-test/design_group.mat',
                                                                                    config['main_dir'] + config['seed2vox_dir'] + '/2_second_level/fsl_designs/2paired-t-test/design_group.con',
                                                                                    config['main_dir'] + config['seed2vox_dir'] + '/2_second_level/fsl_designs/2paired-t-test/design_group.grp',
                                                                                mask_target))

In [ ]:
input='/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/masks/spinalcord_C1C8-L.nii.gz'
mask='/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/masks/spinalcord_GM-cord.nii.gz'
output='/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/masks/spinalcord_GM-L.nii.gz'
run_proc('fslmaths {} -mas {} {}'.format(input,mask,output))